<a href="https://colab.research.google.com/github/Viswanth13/MMTransformer_VT/blob/main/MMT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi Modal Model Transformer

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import torch
import torchvision.transforms as transforms
from transformers import AutoTokenizer
from PIL import Image
import numpy as np

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"viswantht13","key":"8773d7a85fe5b4e56f8e0fbc1513858b"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c petfinder-adoption-prediction

 99% 1.93G/1.94G [00:29<00:00, 25.7MB/s]
100% 1.94G/1.94G [00:30<00:00, 68.9MB/s]


In [5]:
!unzip petfinder-adoption-prediction.zip -d petfinder_data

Streaming output truncated to the last 5000 lines.
  inflating: petfinder_data/train_sentiment/a89bfa0aa.json  
  inflating: petfinder_data/train_sentiment/a89d4e8f3.json  
  inflating: petfinder_data/train_sentiment/a89f8b241.json  
  inflating: petfinder_data/train_sentiment/a89fd1f1e.json  
  inflating: petfinder_data/train_sentiment/a8a1d4151.json  
  inflating: petfinder_data/train_sentiment/a8a2aecc3.json  
  inflating: petfinder_data/train_sentiment/a8a3c4f49.json  
  inflating: petfinder_data/train_sentiment/a8a8f95f6.json  
  inflating: petfinder_data/train_sentiment/a8b358af2.json  
  inflating: petfinder_data/train_sentiment/a8b9d57b4.json  
  inflating: petfinder_data/train_sentiment/a8ba4dfa1.json  
  inflating: petfinder_data/train_sentiment/a8c19a325.json  
  inflating: petfinder_data/train_sentiment/a8c336257.json  
  inflating: petfinder_data/train_sentiment/a8c555240.json  
  inflating: petfinder_data/train_sentiment/a8cdba660.json  
  inflating: petfinder_data/train_

In [6]:
import pandas as pd

# Load train.csv
df = pd.read_csv("petfinder_data/train/train.csv")

# Display the first few rows
df.head()


,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [7]:
TEXT_COLUMN = "Description"
IMAGE_COLUMN = "PetID"
NUMERICAL_COLUMNS = ["Age", "Fee", "PhotoAmt"]
LABEL_COLUMN = "AdoptionSpeed"

In [8]:
import torch
from transformers import AutoTokenizer

# Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_text(text):
    if pd.isna(text):  # Handle missing descriptions
        text = ""
    return tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Example usage
text_tokens = tokenize_text(df[TEXT_COLUMN].iloc[0])
print("Tokenized Text:", text_tokens)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenized Text: {'input_ids': tensor([[  101,  9152, 11362,  2003,  1037,  1017,  1009,  3204,  2214,  3608,
          1997, 10140,  2791,  1012,  2002,  2003, 18114,  1998, 18378,  1012,
          1045, 10148,  1037,  3232,  1997,  8870,  1037,  2261,  2706,  3283,
          2021,  2071,  2025,  2131,  2068, 11265, 19901,  2098,  1999,  2051,
          2004,  1996,  9349,  2001,  3929,  5115,  1012,  1996,  2765,  2001,
          2023,  2210, 14433,  1012,  1045,  2079,  2025,  2031,  2438,  2686,
          1998,  5029,  2000,  2729,  2005,  2062,  8870,  1999,  2026,  4398,
          1012,  2559,  2005,  3625,  2111,  2000,  2202,  2058,  9152, 11362,
          1005,  1055,  2729,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 

In [9]:
import torchvision.transforms as transforms
from PIL import Image
import os

# Define image directory path
IMAGE_DIR = "petfinder_data/train_images"

# Preprocessing function for images
def preprocess_image(pet_id):
    image_path = os.path.join(IMAGE_DIR, f"{pet_id}-1.jpg")  # Use "-1.jpg" convention
    if not os.path.exists(image_path):  # Handle missing images
        print(f"Image not found for {pet_id}")
        return None

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0)

# Example usage
image_tensor = preprocess_image(df[IMAGE_COLUMN].iloc[0])
if image_tensor is not None:
    print("Image Shape:", image_tensor.shape)


Image Shape: torch.Size([1, 3, 224, 224])


In [10]:
import numpy as np
import torch

# Convert numerical features to tensor with explicit type conversion
def get_numerical_tensor(row):
    # Convert to float, replace NaNs with 0 (or any suitable default)
    numerical_values = row[NUMERICAL_COLUMNS].astype(float).fillna(0).values
    return torch.tensor(numerical_values, dtype=torch.float32)

# Example usage
numerical_tensor = get_numerical_tensor(df.iloc[0])
print("Numerical Data:", numerical_tensor)

Numerical Data: tensor([  3., 100.,   1.])


In [11]:
def process_sample(row):
    text_tokens = tokenize_text(row[TEXT_COLUMN])
    image_tensor = preprocess_image(row[IMAGE_COLUMN])
    numerical_tensor = get_numerical_tensor(row)

    return text_tokens, image_tensor, numerical_tensor

# Example usage
text_tokens, image_tensor, numerical_tensor = process_sample(df.iloc[0])
print("Processed Sample:")
print("Text Tokens:", text_tokens)
print("Image Tensor Shape:", image_tensor.shape if image_tensor is not None else "No Image")
print("Numerical Features:", numerical_tensor)


Processed Sample:
Text Tokens: {'input_ids': tensor([[  101,  9152, 11362,  2003,  1037,  1017,  1009,  3204,  2214,  3608,
          1997, 10140,  2791,  1012,  2002,  2003, 18114,  1998, 18378,  1012,
          1045, 10148,  1037,  3232,  1997,  8870,  1037,  2261,  2706,  3283,
          2021,  2071,  2025,  2131,  2068, 11265, 19901,  2098,  1999,  2051,
          2004,  1996,  9349,  2001,  3929,  5115,  1012,  1996,  2765,  2001,
          2023,  2210, 14433,  1012,  1045,  2079,  2025,  2031,  2438,  2686,
          1998,  5029,  2000,  2729,  2005,  2062,  8870,  1999,  2026,  4398,
          1012,  2559,  2005,  3625,  2111,  2000,  2202,  2058,  9152, 11362,
          1005,  1055,  2729,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 

In [12]:
import torch
import torch.nn as nn
from transformers import AutoModel
import torchvision.models as models

In [13]:
# Define the multimodal model for PetFinder
class MultiModalPetfinderModel(nn.Module):
    def __init__(self, text_model_name, image_model_name, numerical_input_size, output_classes):
        super(MultiModalPetfinderModel, self).__init__()
        # Text encoder: e.g., BERT-base uncased; hidden size is typically 768
        self.text_model = AutoModel.from_pretrained(text_model_name)

        # Image encoder: load a pre-trained model (e.g., resnet18) and replace its final layer with an Identity layer.
        # The default output of resnet18 is 512-dimensional.
        self.image_encoder = models.__dict__[image_model_name](pretrained=True)
        self.image_encoder.fc = nn.Identity()  # Remove the classification head

        # Numerical feature processing: map the numerical features to a 128-dimensional embedding.
        self.fc_numeric = nn.Linear(numerical_input_size, 128)

        # Final classifier: concatenate text (768-dim), image (512-dim), and numerical (128-dim) features.
        combined_dim = 768 + 512 + 128
        self.fc_combined = nn.Linear(combined_dim, output_classes)

    def forward(self, text_tokens, image_tensor, numerical_data):
        # Process text: get the CLS token representation from BERT's last hidden state.
        text_features = self.text_model(**text_tokens).last_hidden_state[:, 0, :]  # shape: (batch, 768)

        # Process image: pass image tensor through the image encoder.
        image_features = self.image_encoder(image_tensor)  # shape: (batch, 512)

        # Process numerical data: pass through a fully connected layer.
        numeric_features = self.fc_numeric(numerical_data)  # shape: (batch, 128)

        # Concatenate the features along the feature dimension.
        combined_features = torch.cat((text_features, image_features, numeric_features), dim=1)

        # Pass through the final classifier.
        output = self.fc_combined(combined_features)  # shape: (batch, output_classes)
        return output

In [14]:
# Determine the number of output classes from the target column.
output_classes = len(df[LABEL_COLUMN].unique())

In [15]:
# Create the model instance.
model = MultiModalPetfinderModel(
    text_model_name="bert-base-uncased",
    image_model_name="resnet18",
    numerical_input_size=len(NUMERICAL_COLUMNS),
    output_classes=output_classes
)

print("Model Ready:")
print(model)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 107MB/s]


Model Ready:
MultiModalPetfinderModel(
  (text_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(

In [16]:
model_save_path = "multi_modal_model.pth"
torch.save(model.state_dict(), model_save_path)
print("Model saved successfully")

# Verify that the model file exists
if os.path.exists(model_save_path):
    print("Model file found at:", model_save_path)
else:
    print("Model file is missing. Please train and save it again.")

Model saved successfully
Model file found at: multi_modal_model.pth


In [17]:
!pip install streamlit torch torchvision pillow pyngrok sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [18]:
# Load the state dictionary
state_dict = torch.load("multi_modal_model.pth", map_location=torch.device("cpu"))

# Print model's expected keys
print("Expected model keys:", model.state_dict().keys())

# Print saved state_dict keys
print("Loaded state_dict keys:", state_dict.keys())

<ipython-input-18-d5e5ec477dfa>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("multi_modal_model.pth", map_location=torch.device("cpu"))


Expected model keys: odict_keys(['text_model.embeddings.word_embeddings.weight', 'text_model.embeddings.position_embeddings.weight', 'text_model.embeddings.token_type_embeddings.weight', 'text_model.embeddings.LayerNorm.weight', 'text_model.embeddings.LayerNorm.bias', 'text_model.encoder.layer.0.attention.self.query.weight', 'text_model.encoder.layer.0.attention.self.query.bias', 'text_model.encoder.layer.0.attention.self.key.weight', 'text_model.encoder.layer.0.attention.self.key.bias', 'text_model.encoder.layer.0.attention.self.value.weight', 'text_model.encoder.layer.0.attention.self.value.bias', 'text_model.encoder.layer.0.attention.output.dense.weight', 'text_model.encoder.layer.0.attention.output.dense.bias', 'text_model.encoder.layer.0.attention.output.LayerNorm.weight', 'text_model.encoder.layer.0.attention.output.LayerNorm.bias', 'text_model.encoder.layer.0.intermediate.dense.weight', 'text_model.encoder.layer.0.intermediate.dense.bias', 'text_model.encoder.layer.0.output.dens

In [19]:
%%writefile app.py

import streamlit as st
import torch
import torch.nn as nn
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from transformers import AutoTokenizer, AutoModel
from collections import OrderedDict

# ---------------------------
# Define the multimodal model for PetFinder
# ---------------------------
class MultiModalPetfinderModel(nn.Module):
    def __init__(self, text_model_name, image_model_name, numerical_input_size, output_classes):
        super(MultiModalPetfinderModel, self).__init__()
        # Text encoder: e.g., BERT-base uncased; hidden size is typically 768
        self.text_model = AutoModel.from_pretrained(text_model_name)  # Changed from text_encoder to text_model

        # Image encoder: load a pre-trained model (e.g., resnet18) and replace its final layer with an Identity layer.
        # The default output of resnet18 is 512-dimensional.
        self.image_encoder = models.__dict__[image_model_name](pretrained=True)
        self.image_encoder.fc = nn.Identity()  # Remove the classification head

        # Numerical feature processing: map the numerical features to a 128-dimensional embedding.
        self.fc_numeric = nn.Linear(numerical_input_size, 128)

        # Final classifier: concatenate text (768-dim), image (512-dim), and numerical (128-dim) features.
        combined_dim = 768 + 512 + 128
        self.fc_combined = nn.Linear(combined_dim, output_classes)

    def forward(self, text_tokens, image_tensor, numerical_data):
        # Process text: get the CLS token representation from BERT's last hidden state.
        text_features = self.text_model(**text_tokens).last_hidden_state[:, 0, :]  # shape: (batch, 768)

        # Process image: pass image tensor through the image encoder.
        image_features = self.image_encoder(image_tensor)  # shape: (batch, 512)

        # Process numerical data: pass through a fully connected layer.
        numeric_features = self.fc_numeric(numerical_data)  # shape: (batch, 128)

        # Concatenate the features along the feature dimension.
        combined_features = torch.cat((text_features, image_features, numeric_features), dim=1)

        # Pass through the final classifier.
        output = self.fc_combined(combined_features)  # shape: (batch, output_classes)
        return output

# ---------------------------
# Define dataset column names for PetFinder
# ---------------------------
TEXT_COLUMN = "Description"
IMAGE_COLUMN = "PetID"  # (In the app, you will upload an image file)
NUMERICAL_COLUMNS = ["Age", "Fee", "PhotoAmt"]
LABEL_COLUMN = "AdoptionSpeed"  # (Typically values 0, 1, 2, 3, or 4)

# ---------------------------
# Initialize Streamlit app
# ---------------------------
st.title("PetFinder Multi-Modal Prediction")

# ---------------------------
# Load the trained model
# ---------------------------
# For PetFinder: 3 numerical features and 5 possible adoption speeds (0-4)
model = MultiModalPetfinderModel(
    text_model_name="bert-base-uncased",
    image_model_name="resnet18",
    numerical_input_size=len(NUMERICAL_COLUMNS),
    output_classes=5
)
# Load the model weights (ensure that multi_modal_model.pth exists in your working directory)
state_dict = torch.load("multi_modal_model.pth", map_location=torch.device("cpu"))
# model.load_state_dict(state_dict, strict=False)

new_state_dict = OrderedDict()

for key, value in state_dict.items():
    new_key = key.replace("text_encoder", "text_model")
    new_state_dict[new_key] = value

model.load_state_dict(new_state_dict, strict=False)
model.eval()
# ---------------------------
# Load tokenizer for text processing
# ---------------------------
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# ---------------------------
# Input fields for PetFinder data
# ---------------------------
description = st.text_area("Enter Pet Description")
age = st.number_input("Age", min_value=0, max_value=30, value=3, step=1)
fee = st.number_input("Adoption Fee", min_value=0, max_value=1000, value=100, step=1)
photo_amt = st.number_input("Photo Amount", min_value=0, max_value=10, value=1, step=1)
image_file = st.file_uploader("Upload Pet Image", type=["jpg", "png"])

# ---------------------------
# Prediction logic
# ---------------------------
if st.button("Predict"):
    if description and image_file is not None:
        # Save the uploaded image temporarily
        image_path = "uploaded_pet_image.jpg"
        with open(image_path, "wb") as f:
            f.write(image_file.read())

        # Process the text input
        text_tokens = tokenizer(description, return_tensors="pt", padding=True, truncation=True)

        # Process the image input
        image = Image.open(image_path).convert("RGB")
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        image_tensor = transform(image).unsqueeze(0)

        # Process numerical features (ensure shape is [1, 3])
        numerical_data = torch.tensor([[age, fee, photo_amt]], dtype=torch.float32)

        # Get model prediction
        with torch.no_grad():
            prediction = model(text_tokens, image_tensor, numerical_data)
            # For multi-class classification, take the argmax of the output logits
            predicted_speed = torch.argmax(prediction, dim=1).item()

        st.success(f"Predicted Adoption Speed: {predicted_speed}")
    else:
        st.warning("Please enter a pet description and upload a pet image.")

Writing app.py


In [23]:
!pip install pyngrok
from pyngrok import ngrok

# Authenticate ngrok
ngrok.set_auth_token("2sbM5O5ELNlkZnKhFKkIA560kSm_5ajHjuaspCDzzAbBdeHCD")

# Start ngrok tunnel on port 8501 (as an integer) with HTTP protocol
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
!streamlit run app.py

Streamlit app is live at: NgrokTunnel: "https://cd4e-34-125-82-175.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.82.175:8501

2025-02-07 04:35:31.959064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738902931.983112    5427 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738902931.990212    5427 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be r